In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random, math, time, os, datetime
import importlib

In [55]:
# Run this cell to load/reload modules:
os.chdir("../scripts")

import BTC_KnapFallBack
import BTC_KnapAlg
import BTC_KnapLevAlg
import BTC_Simulation
import BTC_Utils
importlib.reload(BTC_KnapFallBack)
importlib.reload(BTC_KnapAlg)
importlib.reload(BTC_KnapLevAlg)
importlib.reload(BTC_Simulation)
importlib.reload(BTC_Simulation)
importlib.reload(BTC_Utils)
from BTC_KnapFallBack import *
from BTC_KnapAlg import *
from BTC_KnapLevAlg import *
from BTC_Simulation import * 
from BTC_Utils import *

os.chdir("../notebooks")

In [56]:
# Load UTXO and Payment Request Data

df_utxo = pd.read_csv("../data/utxo_sample.csv")
df_cc = pd.read_csv("../data/cc_data.csv")
cc_amts = df_cc['TransactionAmt'].apply(lambda x: USD_to_Satoshi(x,8582)) # Convert USD to Satoshi
cc_amts = cc_amts[cc_amts > 4004] # Ignore amounts <= 4004 in Satoshi

df_utxo = df_utxo[df_utxo['amount'] >= 4004] # Only consider UTXOs larger than 4004 Satoshi
df_pr = df_utxo[df_utxo['amount'] <= df_utxo['amount'].quantile(.95)] # Consider "resonable" UTXOs as payment requests

In [57]:
# Random Sample UTXO data

def RandomSampleUTXOData(num_utxos, num_pay_req, num_high_urg, randomState, payRequestData = 'CC'):
    
    if payRequestData == 'UTXO':
        utxos = np.array(df_utxo['amount'].sample(num_utxos, random_state=randomState))
        pay_req = np.array(df_pr['amount'].sample(num_pay_req, random_state=randomState))
        high_urg = pay_req[:num_high_urg]
    else:
        utxos = np.array(df_utxo['amount'].sample(num_utxos, random_state=randomState))
        pay_req = np.array(cc_amts.sample(num_pay_req, random_state=randomState))
        high_urg = pay_req[:num_high_urg]
    
    return utxos, pay_req, high_urg

In [60]:
knap_sr = pd.read_csv("../simulation-results/KnapsackSuccessRate_noLev.csv") # Results of prior simulations to get Boost Factor

In [62]:
def GetBoostFactor(feePerByte, payRequestsPerTX, df):
    return 1- df[df['feePerByte'] == feePerByte][df['Pay Requests Per TX'] == payRequestsPerTX]['Knapsack Success Rate'].iloc[0]

In [64]:
# Running the simulation

sims = []
summaries = []

startTimeFull = time.time()

# parameters

numUTXOs = 2500 # Size of UTXO Pool
numPayRequests = 250 # Size of Payment Requests Pool

for numHighUrg in [2,3,5,10]: # Range of values to test for number of pay reqs to process each tx
    for feePerByte in [22, 60, 200, 400, 900]: # Range of values to test fee-per-byte
        for rand in [17, 19, 21, 23, 31, 32, 33, 34, 35, 71]: # Range of random seeds for simulation
            utxos, pay_req, high_urg = RandomSampleUTXOData(numUTXOs, numPayRequests, numHighUrg, rand, payRequestData="CC")
            startTime = time.time()
            sim, summary = BTC_Simulation_KnapLev(utxos, pay_req, numHighUrg, leverage=True, print_details=True, iterations=5, \
                                   fee_per_byte = feePerByte, oneBTC = 8582, \
                                   lev_t1lb = numHighUrg, lev_t1ub = numHighUrg, lev_boost_factor = GetBoostFactor(feePerByte, numHighUrg,knap_sr), \
                                     time_limit_knap = 10)
            sims.append(sim)
            summaries.append(summary)

endTimeFull = time.time()

C:\Users\daniel.diroff\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


/////////////////////////////////////////////////////////////////////
/////////////////////////////////////////////////////////////////////
Iteration : 1
UTXO Pool Size: 2500
Pay Requests Left: 250
/////////////////////////////////////////////////////////////////////
Trying Knapsack Solution:
Problem Status: Infeasible
The problem took 0.4113621711730957 seconds to solve
Sucess: False
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
Input Total: $1201.4267003975274
Payment Requests Processed:
p_0: Value = $97.0
p_1: Value = $1104.0
Output Total: $1201.0
Transaction Fee: $0.42669703999999997
Overpayment: $3.3575274373590945e-06
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasibi

Problem Status: Infeasible
The problem took 0.8962254524230957 seconds to solve
Sucess: False
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
Input Total: $346.7256992569603
Payment Requests Processed:
p_0: Value = $11.299
p_1: Value = $335.00000000000006
Output Total: $346.29900000000004
Transaction Fee: $0.42669703999999997
Overpayment: $2.216960202604532e-06
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasibility Check:
Optimal Number of Inputs Used: False
Sufficient Funds: True
Overpayment Less than Dust: True
Valid Transaction: False
Fall Back Solution: True
/////////////////////////////////////////////////////////////////////
Trying Leverage Solution:
Problem Status: Opt

Problem Status: Infeasible
The problem took 0.535513162612915 seconds to solve
Sucess: False
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
Input Total: $385.37669697439446
Payment Requests Processed:
p_0: Value = $200.00000000000006
p_1: Value = $184.95
Output Total: $384.95000000000005
Transaction Fee: $0.42669703999999997
Overpayment: $-6.560555136665244e-08
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasibility Check:
Optimal Number of Inputs Used: False
Sufficient Funds: False
Overpayment Less than Dust: True
Valid Transaction: False
Fall Back Solution: True
/////////////////////////////////////////////////////////////////////
Trying Leverage Solution:
Problem Status: O

/////////////////////////////////////////////////////////////////////
Using Fallback Solution
/////////////////////////////////////////////////////////////////////
2 Payment Requests Processed via Fall Back Solution
Success
/////////////////////////////////////////////////////////////////////
/////////////////////////////////////////////////////////////////////
/////////////////////////////////////////////////////////////////////
/////////////////////////////////////////////////////////////////////
/////////////////////////////////////////////////////////////////////
Summary Statistics:
/////////////////////////////////////////////////////////////////////
Number of UTXOs Used: 5
Number of Pay Requests Processed: 14
Cost Per Pay Request Processed: $0.23851221428571057
Total Cost to Process Payments: $3.339170999999948
# of Knapsack Successful Solutions: 1
# of Leverage Successful Solutions: 2
# of Fallback Successful Solutions: 2
/////////////////////////////////////////////////////////

...

Problem Status: Optimal
The problem took 0.5814597606658936 seconds to solve
Sucess: True
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
x_903: Value = $1071.62206774
Input Total: $1071.62206774
Payment Requests Processed:
p_0: Value = $171.0
p_1: Value = $75.0
p_2: Value = $117.0
p_3: Value = $5.0
p_4: Value = $150.0
p_5: Value = $134.13299999999998
p_6: Value = $34.5
p_7: Value = $98.95000000000002
p_8: Value = $32.563
p_9: Value = $214.00000000000003
Output Total: $1032.1460000000002
Transaction Fee: $38.464524
Overpayment: $1.0115437399998912
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasibility Check:
Optimal Number of Inputs Used: True
Sufficient Funds: True
Overpayme

Problem Status: Infeasible
The problem took 0.4867124557495117 seconds to solve
Sucess: False
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
Input Total: $1012.488525625524
Payment Requests Processed:
p_0: Value = $166.99
p_1: Value = $63.605
p_2: Value = $7.579000000000001
p_3: Value = $209.95000000000002
p_4: Value = $49.0
p_5: Value = $97.0
p_6: Value = $34.0
p_7: Value = $43.95
p_8: Value = $92.0
p_9: Value = $209.95000000000002
Output Total: $974.0240000000001
Transaction Fee: $38.464524
Overpayment: $1.6255237808242783e-06
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasibility Check:
Optimal Number of Inputs Used: False
Sufficient Funds: True
Overpayment Less than Dust

Problem Status: Infeasible
The problem took 0.5236148834228516 seconds to solve
Sucess: False
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
Input Total: $1174.734519167456
Payment Requests Processed:
p_0: Value = $57.95
p_1: Value = $104.47
p_2: Value = $97.0
p_3: Value = $167.95
p_4: Value = $57.95
p_5: Value = $25.000000000000007
p_6: Value = $470.0
p_7: Value = $67.95
p_8: Value = $59.00000000000001
p_9: Value = $29.000000000000007
Output Total: $1136.2700000000002
Transaction Fee: $38.464524
Overpayment: $-4.832544135762168e-06
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasibility Check:
Optimal Number of Inputs Used: False
Sufficient Funds: False
Overpayment Less than

Problem Status: Infeasible
The problem took 0.4777352809906006 seconds to solve
Sucess: False
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
Input Total: $719.2835202319244
Payment Requests Processed:
p_0: Value = $87.95000000000002
p_1: Value = $50.000000000000014
p_2: Value = $107.95
p_3: Value = $107.95
p_4: Value = $7.069
p_5: Value = $35.95
p_6: Value = $82.0
p_7: Value = $117.0
p_8: Value = $49.0
p_9: Value = $35.95
Output Total: $680.819
Transaction Fee: $38.464524
Overpayment: $-3.7680755405305355e-06
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasibility Check:
Optimal Number of Inputs Used: False
Sufficient Funds: False
Overpayment Less than Dust: True
Valid Transa

Problem Status: Optimal
The problem took 0.6841871738433838 seconds to solve
Sucess: True
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Transaction Details:
UTXOs used in Transaction:
x_1103: Value = $937.9066123
Input Total: $937.9066123
Payment Requests Processed:
p_0: Value = $117.0
p_1: Value = $159.95000000000002
p_2: Value = $34.5
p_3: Value = $50.000000000000014
p_4: Value = $49.0
p_5: Value = $117.0
p_6: Value = $40.0
p_7: Value = $171.0
p_8: Value = $100.00000000000003
p_9: Value = $59.00000000000001
Output Total: $897.4500000000002
Transaction Fee: $38.464524
Overpayment: $1.9920882999999099
Optimal Number of Inputs: 1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Manual Feasibility Check:
Optimal Number of Inputs Used: True
Sufficient Funds: True


In [65]:
print("The simulations took: " + str(endTimeFull - startTimeFull) + " seconds.")
print("The simulations took: " + str((endTimeFull - startTimeFull)/60) + " minutes.")

The simulations took: 4212.652723312378 seconds.


In [66]:
save_results_path = "../simulation-results/leverage-results.csv" # path where to save csv file with simulation results

results = pd.DataFrame(summaries)
results.to_csv(save_results_path, index=False)